In [ ]:
import pandas as pd
from sklearn import model_selection
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


df = pd.read_csv('./ml-20m/ratings.csv')
X = df[['userId', 'movieId']].values
Y = df[['rating']].values
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X, Y, test_size=0.1)
train_dataset = TensorDataset(torch.LongTensor(train_X), torch.FloatTensor(train_Y))
test_dataset = TensorDataset(torch.LongTensor(test_X), torch.FloatTensor(test_Y))
train_loader = DataLoader(train_dataset, batch_size=1024, num_workers=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, num_workers=4)

In [ ]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=100, metavar='N',help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,help='For Saving the current Model')
args = parser.parse_args()
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
from torch import nn
# 行列因子分解の内積をとる代わりにMLPを通して非線形化
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # ユーザー特徴量と商品特徴量をまとめて一つのベクトルにする
        out = torch.cat([user_feature, item_feature], 1)
        # まとめた特徴量ベクトルをMLPに入れる
        # 内積をしないので、dot((n, k),(k, m))の様な特徴量ベクトルの次元をそろえる必要がない
        # バッチノーマリゼーションのようなニューラルネットの訓練のテクニックがそのまま使える
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

In [ ]:
max_user, max_item = X.max(0)
max_user = int(max_user)
max_item = int(max_item)
net = NeuralMatrixFactorization(max_user+1, max_item+1)

In [ ]:
from torch.autograd import Variable as V
from statistics import mean
def mae(x, y):
    # 平均絶対誤差を計算
    return (x - y).abs().mean()

def eval_net(net, loader, score_fn=mae):
    ys = []
    ypreds = []
    for x, y in loader:
        x = V(x)
        ys.append(y)
        ypred = net(x).data
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

In [ ]:
def train_model(net, dataloders_dict, criterion, optimizer, num_epochs):

    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    #初期設定
    #GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス", device)
    #モデルをGPUへ
    net.to(device)
    #ネットワークがある程度固定であれば高速化させる
    torch.backends.cudnn.benchmark = True
    #epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-----------------------------------')
        #epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() #モデルを訓練モードに
            else:
                net.eval() #モデルを検証モードに
            epoch_loss = 0.0 #epochの損失0
            epoch_corrects = 0 #epochの正解数
            print(dataloders_dict[phase])
            #データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                #optimizerを初期化
                optimizer.zero_grad()
                #順伝搬(forward)計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)#損失を計算
                    _, preds = torch.max(outputs, 1) #ラベルを予測
                    #訓練時はバックプロパゲーション

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    #イテレーション結果の計算
                    #lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    #正解の合計数を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            #epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloders_dict[phase].dataset)

            print('{} Loss:{:.4f} Acc: {:.4f}'.format(phase, epoch_loss,epoch_acc))
            if phase == 'train':
                train_acc_list.append(epoch_acc)
                train_loss_list.append(epoch_loss)
            else:
                val_acc_list.append(epoch_acc)
                val_loss_list.append(epoch_loss)

        if(((epoch + 1) % 10) == 0):
            save_path = './weights_' + str(epoch) + 'epoch.pth'
            torch.save(net.state_dict(), save_path)
    return val_loss_list,train_loss_list, val_acc_list, train_acc_list